In [1]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

### Prepairing the Data

In [2]:
train_data = np.load("clean_train.npy")
test_data = np.load("clean_test.npy")

In [3]:
train_df = pd.DataFrame(list(train_data))
test_df = pd.DataFrame(list(test_data))

In [4]:
train_df.describe()

,A,S,SS,age,country_rank,favourite_beatmapset_count,follower_count,hit_accuracy,level,maximum_combo,play_count,play_time,post_count,pp,ranked_and_approved_beatmapset_count,ranked_score,replays_watched_by_others,total_hits,total_score
count,6976.000000,6976.000000,6976.000000,4878.000000,6945.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6.976000e+03,6.976000e+03,6.976000e+03,6.976000e+03
mean,774.774226,855.727924,124.393779,20.443214,371.202016,21.158257,228.605361,98.389156,97.477208,3228.451405,57030.484088,766.744755,50.010321,6560.831422,0.242546,1.605114e+10,5.853665e+03,1.148344e+07,8.841395e+10
std,448.361125,712.447531,325.600637,12.458928,1321.261574,26.965613,1406.944825,1.603457,12.415682,1312.364611,34478.559485,433.749903,323.209984,1106.749009,3.392499,1.349912e+10,9.635305e+04,6.676292e+06,7.048663e+10
min,-1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,39.822400,4.000000,11.000000,3.000000,0.068889,0.000000,5447.000000,0.000000,1.662400e+05,0.000000e+00,1.368000e+03,4.623100e+05
25%,467.000000,449.000000,23.000000,17.000000,48.000000,2.000000,34.000000,98.220150,100.000000,2274.000000,35024.000000,500.540139,0.000000,5763.000000,0.000000,8.052893e+09,2.000000e+01,7.234938e+06,4.556414e+10
50%,700.000000,672.000000,49.000000,19.000000,154.000000,9.000000,75.000000,98.708800,100.000000,3017.000000,51939.000000,708.905972,2.000000,6228.000000,0.000000,1.286739e+10,7.400000e+01,1.031621e+07,7.100375e+10
75%,993.000000,1047.000000,113.000000,21.000000,374.000000,31.000000,160.000000,99.042800,100.000000,3936.000000,72886.250000,963.681875,11.000000,6989.000000,0.000000,2.029199e+10,3.622500e+02,1.454376e+07,1.110794e+11
max,5847.000000,15734.000000,13388.000000,103.000000,44563.000000,100.000000,73342.000000,100.000000,107.000000,15464.000000,353971.000000,5159.530833,16091.000000,14296.000000,220.000000,2.695437e+11,6.472658e+06,5.551982e+07,7.602120e+11


In [5]:
# train_df.isnull().sum()

In [6]:
use_list = ['country_rank','play_count','age', 'ranked_score','pp']

In [7]:
train_df = pd.DataFrame(train_df, columns=use_list)

In [8]:
use_list = ['username','country_rank','play_count', 'age', 'ranked_score']

In [9]:
test_df = pd.DataFrame(test_df, columns=use_list)

In [10]:
test_df.isnull().sum()

username          0
country_rank     19
play_count        0
age             907
ranked_score      0
dtype: int64

In [11]:
avg_rank = test_df['country_rank'].mean(axis=0)
train_df['country_rank'].replace(np.nan, avg_rank, inplace= True)
test_df['country_rank'].replace(np.nan, avg_rank, inplace= True)

In [12]:
avg_age = test_df['age'].mean(axis=0)
train_df['age'].replace(np.nan, avg_age, inplace= True)
test_df['age'].replace(np.nan, avg_age, inplace= True)

In [13]:
for column in train_df.columns:
    if column != 'pp':
        mean = train_df[column].mean()
        std = train_df[column].std()
        train_df[column] = (train_df[column] - mean) / std
        test_df[column] = (test_df[column] - mean) / std

In [14]:
train_df.describe()

,country_rank,play_count,age,ranked_score,pp
count,6.976000e+03,6.976000e+03,6.976000e+03,6.976000e+03,6976.000000
mean,3.259370e-17,3.259370e-17,6.620596e-17,-3.259370e-17,6560.831422
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1106.749009
min,-2.807218e-01,-1.653998e+00,-1.932538e+00,-1.189038e+00,5447.000000
25%,-2.450704e-01,-6.382658e-01,-2.063341e-01,-5.925012e-01,5763.000000
50%,-1.631482e-01,-1.476710e-01,-6.503401e-02,-2.358484e-01,6228.000000
75%,1.454807e-03,4.598732e-01,-1.453361e-02,3.141577e-01,6989.000000
max,3.352130e+01,8.612324e+00,7.945185e+00,1.877845e+01,14296.000000


In [15]:
train_df = train_df[train_df['country_rank'] < 3]

In [16]:
train_df = train_df[train_df['play_count'] < 3]

In [17]:
train_df = train_df[train_df['ranked_score'] < 3]

In [18]:
# train_df = train_df[train_df['follower_count'] < 3]

In [19]:
labels = train_df.pp
l_mean = labels.mean()
l_std = labels.std()
labels = (labels - l_mean)/l_std

In [20]:
train_df.drop(['pp'], axis=1, inplace=True)

In [21]:
train_df.describe()

,country_rank,play_count,age,ranked_score
count,6746.000000,6746.000000,6746.000000,6746.000000
mean,-0.061979,-0.056319,-0.004480,-0.077074
std,0.301157,0.862750,1.004727,0.751470
min,-0.280722,-1.653998,-1.932538,-1.189019
25%,-0.244312,-0.638766,-0.206334,-0.592770
50%,-0.162390,-0.164479,-0.065034,-0.248595
75%,-0.000821,0.416593,-0.014534,0.267582
max,2.952931,2.994224,7.945185,2.982285


### processing data

In [22]:
X = np.array(train_df)

In [23]:
y = np.array(labels)

In [24]:
test_d = np.array(test_df)

In [25]:
# a[:,1:] without username

In [26]:
X.shape, y.shape, test_d.shape

((6746, 4), (6746,), (2994, 5))

In [27]:
def estimate_model(X, y,Lambda = 0):
    
    # X transpose
    Xtranspose = np.matrix.transpose(X)
    
    # Identity matrix (number of parameters is the dimension)
    Identity = np.identity(len(X[1,:]))
    
    # We don't add penalty to intercept
    Identity[0,0] = 0
    
    # Closed form solution is BetaHat = inv(X'X + Lambda*I)*X'y
    # Estimate model parameters (if Lambda = 0, we get standard square loss function result)
    BetaHat = np.dot(np.linalg.inv(np.add(np.dot(Xtranspose,X),Lambda*Identity)),np.dot(Xtranspose,y))
    
    return BetaHat

In [28]:
folds = [X[i::5] for i in range(5)]
folds_y = [y[i::5] for i in range(5)]

In [29]:
results = {}
for d in range(1, 7):
    loss = 0
    for i in range(5):
        train = np.array(folds[i])
        test = np.array(folds_y[i])
        
        x_train, x_test, y_train, y_test = train_test_split(train, test, test_size=0.2)
        
        poly = PolynomialFeatures(d)
        X_train_poly = poly.fit_transform(x_train)
        X_test_poly = poly.fit_transform(x_test)
        
        betas = estimate_model(X_train_poly, y_train)
        predicted = [np.array(t).dot(betas) for t in X_test_poly]
        loss += sum([(predicted[index] - y_test[index])**2 for index in range(len(predicted))])/len(predicted)
    results[d] = loss
    print ("mean loss: {} , degre: {}".format(loss, d))


mean loss: 3.9535424776437407 , degre: 1
mean loss: 2.9993344960839137 , degre: 2
mean loss: 2.8166519985037506 , degre: 3
mean loss: 5.5073090490091055 , degre: 4
mean loss: 481.76013398861573 , degre: 5
mean loss: 126185.71365105268 , degre: 6


In [30]:
min_degre = min(results, key=results.get) 

### Final Model Reg

In [31]:
results = {}
for l in np.arange(0, 1, 0.1):        
    mean_tmp = []
    loss = 0
    for i in range(5):
        train = np.array(folds[i])
        test = np.array(folds_y[i])

        x_train, x_test, y_train, y_test = train_test_split(train, test, test_size=0.2)

        poly = PolynomialFeatures(min_degre)
        X_train_poly = poly.fit_transform(x_train)
        X_test_poly = poly.fit_transform(x_test)

        betas = estimate_model(X_train_poly, y_train, l)
        predicted = [np.array(t).dot(betas) for t in X_test_poly]
        loss += sum([(predicted[index] - y_test[index])**2 for index in range(len(predicted))])/len(predicted)
    results[l] = loss
    print ("lambda: {:.1f}, mean loss: {}".format(l, loss))

lambda: 0.0, mean loss: 3.1654955509506553
lambda: 0.1, mean loss: 3.164237732124458
lambda: 0.2, mean loss: 3.4222273422116793
lambda: 0.3, mean loss: 3.2663277249420504
lambda: 0.4, mean loss: 3.1215906496093933
lambda: 0.5, mean loss: 2.9349090824018864
lambda: 0.6, mean loss: 3.252623982575904
lambda: 0.7, mean loss: 3.1309915907619827
lambda: 0.8, mean loss: 2.882195022842292
lambda: 0.9, mean loss: 3.5854535169866137


In [32]:
min_lambda = min(results, key=results.get) 

In [33]:
min_lambda

0.8

In [34]:
poly = PolynomialFeatures(min_degre)

In [35]:
X_poly = poly.fit_transform(X)

In [36]:
test_poly = poly.fit_transform(test_d[:,1:]) # without usernames

In [37]:
B = estimate_model(X_poly, y, min_lambda)

In [38]:
predict = test_poly.dot(B)

In [39]:
predict = predict * l_std + l_mean

In [40]:
result = dict(zip(test_d[:,0], predict))

In [41]:
print("lambda: {}, degre: {}".format(min_lambda, min_degre))

lambda: 0.8, degre: 3


In [42]:
import json
json.dump(result, open("karen_ghandilyan.json", "w"))